# Imports

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn import datasets

In [2]:
from project_lib import Project
project = Project.access()

import os
import requests
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "token": os.environ["USER_ACCESS_TOKEN"],
    "instance_id" : "openshift",
    "url": "xxx",
    "version": "3.5"
}

client = APIClient(wml_credentials)
client.spaces.list(limit=5)
SPACE_ID = "xxx"
PROJECT_ID = "xxx"

------------------------------------  ------------------------  ------------------------
ID                                    NAME                      CREATED
bc23fa58-70c1-4708-b0d5-a40713f710e2  CPD JumpStart Deployment  2021-12-01T03:48:52.903Z
------------------------------------  ------------------------  ------------------------


# Build model

In [3]:
iris = datasets.load_iris()
X = pd.DataFrame(iris["data"], columns=iris["feature_names"])
y = iris["target"]
clf = svm.SVC(kernel="linear", C=0.01).fit(X, y)

In [4]:
# Give the model a name for example and add a suffix using your first name, last name and 4 random numbers, i.e, (Iris SVM Model_RandyP-8650)
MODEL_NAME = "Iris SVM Model_RandyP-8650"
client.set.default_project(PROJECT_ID)

for model in client.repository.get_model_details()["resources"]:
    if MODEL_NAME == model["metadata"]["name"]:
        client.repository.delete(model["metadata"]["id"])
meta_props = {
    client.repository.ModelMetaNames.NAME: MODEL_NAME,
    client.repository.ModelMetaNames.TYPE: "scikit-learn_0.23",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_uid_by_name("default_py3.7_opence"),
}
model_details = client.repository.store_model(model=clf, meta_props=meta_props)

# Deploy model

In [ ]:
# Give the deployment a name for example and add a suffix using your first name, last name and 4 random numbers, i.e, (Iris SVM Deployment_RandyP-8650)
DEPLOYMENT_NAME = "Iris SVM Deployment_RandyP-8650"
client.set.default_space(SPACE_ID)

for deployment in client.deployments.get_details()["resources"]:
    if DEPLOYMENT_NAME in deployment["metadata"]["name"]:
        client.deployments.delete(deployment["metadata"]["id"])
        
for model in client.repository.get_model_details()["resources"]:
    if MODEL_NAME == model["metadata"]["name"]:
        client.repository.delete(model["metadata"]["id"])
        
meta_props = {
    client.repository.ModelMetaNames.NAME: MODEL_NAME,
    client.repository.ModelMetaNames.TYPE: "scikit-learn_0.23",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_uid_by_name("default_py3.7_opence"),
}
model_details = client.repository.store_model(model=clf, meta_props=meta_props)
model_uid = client.repository.get_model_uid(model_details)

meta_props = {client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME + "-online", client.deployments.ConfigurationMetaNames.ONLINE: {}}
deployment_details_online = client.deployments.create(model_uid, meta_props=meta_props)

meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME + "-batch",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { "name": "S", "num_nodes": 1 }
}
deployment_details_batch = client.deployments.create(model_uid, meta_props=meta_props)

deployment_uid_online = client.deployments.get_uid(deployment_details_online)
deployment_uid_batch = client.deployments.get_uid(deployment_details_batch)

In [7]:
client.repository.list_models()
client.deployments.list()

------------------------------------  --------------  ------------------------  -----------------
ID                                    NAME            CREATED                   TYPE
b9989b13-0973-40f5-aba5-96e313eb232c  Iris SVM Model  2021-12-04T18:36:12.002Z  scikit-learn_0.23
------------------------------------  --------------  ------------------------  -----------------
------------------------------------  --------------------------  ------  ------------------------
GUID                                  NAME                        STATE   CREATED
ec0fcfdb-23e2-43cb-a14f-94aa3aec9af3  Iris SVM Deployment-online  failed  2021-12-04T18:36:13.190Z
------------------------------------  --------------------------  ------  ------------------------


# Score model (online)

In [ ]:
payload = {"input_data": [{"values": X.head().values.tolist(),}]}
result = client.deployments.score(deployment_uid_online, payload)
result

# Score model (batch)

In [ ]:
def upload_asset(filename):
    for asset in client.data_assets.get_details()["resources"]:
        if filename in asset["metadata"]["name"]:
            client.data_assets.delete(asset["metadata"]["guid"])
    asset_details = client.data_assets.create(name=filename, file_path=filename)
    return asset_details

input_filename = "input.csv"
X.to_csv(input_filename, index=False)
# project.save_data("data.csv", pd.DataFrame(X).to_csv(index=False))

output_uid = upload_asset(output_filename)["metadata"]["guid"]
input_uid = upload_asset(input_filename)["metadata"]["guid"]

In [ ]:
payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA_REFERENCES: [
        {
            "connection": {},
            "location": {"href": f"/v2/assets/{input_uid}?space_id={SPACE_ID}"},
            "type": "data_asset"
        }
    ],
    client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
        "connection": {},
        "location": {"name": "output.csv"},
        "type": "data_asset"
    },
}
job = client.deployments.create_job(deployment_uid_batch, meta_props=payload)
job_uid = client.deployments.get_job_uid(job)


In [ ]:
client.deployments.get_job_status(job_uid)
print()
client.deployments.get_job_details(job_uid)

In [ ]:
asset_dict = { asset["metadata"]["name"]: asset["metadata"]["guid"] for asset in client.data_assets.get_details()["resources"] }
asset_dict

In [ ]:
client.data_assets.download(asset_dict["output.csv"], "output.csv")